In [8]:
import cv2
from skimage import io, img_as_float
from skimage.filters import rank
from skimage.morphology import disk
import numpy as np
face_results = []
keyp=[]
# Crear un objeto SIFT
sift = cv2.xfeatures2d.SIFT_create()
def detect_faces(r):
    image = cv2.imread(r)
    keypoints, descriptors = sift.detectAndCompute(image, None)
    keyp=len(keypoints)
    return keyp
def evaluate_resolucion(a):
    img = cv2.imread(a)
    height, width, channels = img.shape
    if height < 100 or width < 100:
        return True
    return False

def evaluate_brillo(a)
    img = cv2.imread(a)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    mean_brightness = v.mean()
    return mean_brightness

def evaluate_contraste(a):
    img = cv2.imread(a)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    mean_contrast = cv2.Laplacian(gray, cv2.CV_64F).var()
    return mean_contrast

def evaluate_entropia(a):
    im1 = img_as_float(io.imread(a, as_gray=True))
    radius = 15
    selem = disk(radius)
    entropy = rank.entropy(im1, selem=selem)
    mean_entropy = np.mean(entropy)
    return mean_entropy

def difuminacion(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return laplacian_var

In [9]:
import cv2
from skimage import io, img_as_float
from skimage.filters import rank
from skimage.morphology import disk
import numpy as np
lista_fallas=[]
imagenes=[]
carpeta_base1=[]
n1=[]
puntosdet=[]
brillo=[]
contr=[]
reso=[]
entro=[]
difu=[]

def evaluate_image_qualitys(a):
    if evaluate_resolucion(a):
        return False
    if evaluate_brillo(a) < 14:
        return False
    if evaluate_contraste(a) < 3.5:
        return False
    if evaluate_entropia(a) < 3:
        return False
    if detect_faces(a)<=3:
        return False
    if  difuminacion(a) < 5.7:
        return False
    # Si todos los criterios se cumplen, la imagen es de buena calidad
    return True


In [10]:
import cv2
from skimage.feature import hog
from sklearn.metrics.pairwise import cosine_similarity

def sim_coseno(ruta_imagen1, ruta_imagen2):
    image1 = cv2.imread(ruta_imagen1)
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

    image2 = cv2.imread(ruta_imagen2)
    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

    # Calcular el descriptor HOG para la imagen 1
    fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
                          cells_per_block=(1, 1), visualize=True, multichannel=True, channel_axis=2)

    # Calcular el descriptor HOG para la imagen 2
    fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
                          cells_per_block=(1, 1), visualize=True, multichannel=True, channel_axis=2)

    # Calcular la similitud del coseno entre los descriptores
    similitud = cosine_similarity(fd1.reshape(1, -1), fd2.reshape(1, -1))

    return similitud[0][0]

In [11]:
# PARA LA DETECCION DE FALLAS
import os
import pandas as pd
import numpy as np
#directorio_inicial1 = 'D:\sentirobots\Datasets\AFEW\Datos\Train_AFEW\AlignedFaces_LBPTOP_Points\AlignedFaces_LBPTOP_Points\Ejercicio entropia' # Carpeta actual
directorio_inicial1 = 'D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW' # Carpeta actual
#directorio_inicial1 = 'D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\Val\Faces' # Carpeta actual
pasa=[]
archivos1 = []
carpeta_base1=[]
puntos1=[]
filtro1=[]
file1=[]
r=[]
b=[]
co=[]
e=[]
f=[]
d=[]

for raiz, carpetas, archivos_en_carpeta in os.walk(directorio_inicial1):
    for archivo in archivos_en_carpeta:
        if os.path.isfile(os.path.join(raiz, archivo)):
            ruta_completa1 = os.path.join(raiz, archivo)
            archivos1.append(ruta_completa1)
            r.append(evaluate_resolucion(ruta_completa1))
            b.append(evaluate_brillo(ruta_completa1))
            co.append(evaluate_contraste(ruta_completa1))
            e.append(evaluate_entropia(ruta_completa1))
            f.append(detect_faces(ruta_completa1))
            d.append(difuminacion(ruta_completa1))
            if evaluate_image_qualitys(ruta_completa1):
                pasa.append(1)
                if evaluate_brillo(ruta_completa1)>230 and evaluate_contraste(ruta_completa1)<20:
                    pasa[-1] = 2
                    print(ruta_completa1)
                    print("Transformacion brillo contraste o entropia")
                    print("brillo")
                    print(evaluate_brillo(ruta_completa1))
                    print("contraste")
                    print(evaluate_contraste(ruta_completa1))
                    print("entropia")
                    print(evaluate_entropia(ruta_completa1))
            else:
                pasa.append(0)
                if len(archivos1) > 1:
                    carpeta_anterior = os.path.dirname(archivos1[-2])
                    carpeta_actual = os.path.dirname(archivos1[-1])
                    a1 = os.path.join(archivos1[-2])
                    a2 = os.path.join(archivos1[-1])
                    if carpeta_anterior == carpeta_actual:
                        if pasa[-2] >= 1:
                            if (sim_coseno(a1, a2)>0.9 and difuminacion(ruta_completa1)>4.5 and detect_faces(ruta_completa1)>0) or (detect_faces(ruta_completa1)>9 and sim_coseno(a1, a2)>0.9) or (detect_faces(ruta_completa1)>5 and sim_coseno(a1, a2)>0.9 and difuminacion(ruta_completa1)>5 and evaluate_contraste(ruta_completa1)>4 and evaluate_brillo(ruta_completa1)>14 and evaluate_entropia(ruta_completa1)>4):
                                pasa[-1] = 2
            #detectar_rostros_hog(ruta_completa1)

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:14: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:18: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:14: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:18: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank fil

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:14: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:18: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank fil

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:14: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:18: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank fil

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:14: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:18: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank fil

...

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:14: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:18: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank fil

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:14: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:18: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank fil

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:14: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:18: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank fil

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:14: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd1, hog_image1 = hog(image1, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\205278858.py:18: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd2, hog_image2 = hog(image2, orientations=8, pixels_per_cell=(16, 16),
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank fil

C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = rank.entropy(im1, selem=selem)
C:\Users\Desarrollo\AppData\Local\Temp\ipykernel_14856\1210436803.py:58: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  entropy = rank.entropy(im1, selem=selem)


In [13]:
#Real
archiv={"RutaArchivo": archivos1}
df= pd.DataFrame(archiv)
df["Filtro"] = pasa
df["resolucion"] = r
df["brillo"] = b
df["contraste"] = co
df["entropia"] = e
df["p rostro"] = f
df["difumin"] = d
df

RutaArchivo  Filtro  resolucion  \
0      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
1      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
2      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
3      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
4      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   
...                                                  ...     ...         ...   
63067  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63068  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63069  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63070  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63071  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   

           brillo   contraste  entropia  p rostro     difumin  
0       46.620056  168.688503  5.680590        47  168.688503  
1       47.367188  169.932763  5.672910        34  169.932763  
2       47.509460  180.040745  5.673102        46  180.040745  
3       95.777161  431.258760  6.025994        69  431.258760  
4      220.340332    0.699585  0.618864         0    0.699585  
...           ...         ...       ...       ...         ...  
63067   67.594421   40.351175  5.329795        31   40.351175  
63068   66.825623   39.392251  5.306691        39   39.392251  
63069   66.991760   32.606774  5.312265        34   32.606774  
63070   66.380615   32.986551  5.344828        32   32.986551  
63071   66.631104   33.802185  5.361588        30   33.802185  

[63072 rows x 8 columns]

In [14]:
# Evaluar si los valores de las columnas cumplen con la condición utilizando un OR
condicion1 = df['resolucion'] == True
condicion2 = df['brillo'] < 14
condicion3 = df['contraste'] < 3.5
condicion4 = df['entropia'] < 3
condicion5 = df['p rostro'] <= 3
condicion6 = df['difumin'] <= 5.7
# Contar cuántas columnas cumplen con las condiciones para cada fila
df['Falla en'] = pd.concat([condicion1, condicion2, condicion3, condicion4, condicion5, condicion6], axis=1).sum(axis=1)
df

RutaArchivo  Filtro  resolucion  \
0      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
1      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
2      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
3      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
4      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   
...                                                  ...     ...         ...   
63067  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63068  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63069  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63070  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63071  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   

           brillo   contraste  entropia  p rostro     difumin  Falla en  
0       46.620056  168.688503  5.680590        47  168.688503         0  
1       47.367188  169.932763  5.672910        34  169.932763         0  
2       47.509460  180.040745  5.673102        46  180.040745         0  
3       95.777161  431.258760  6.025994        69  431.258760         0  
4      220.340332    0.699585  0.618864         0    0.699585         4  
...           ...         ...       ...       ...         ...       ...  
63067   67.594421   40.351175  5.329795        31   40.351175         0  
63068   66.825623   39.392251  5.306691        39   39.392251         0  
63069   66.991760   32.606774  5.312265        34   32.606774         0  
63070   66.380615   32.986551  5.344828        32   32.986551         0  
63071   66.631104   33.802185  5.361588        30   33.802185         0  

[63072 rows x 9 columns]

In [15]:
df.describe()

Filtro        brillo     contraste      entropia      p rostro  \
count  63072.000000  63072.000000  63072.000000  63072.000000  63072.000000   
mean       0.957826     64.011046     43.453527      5.228126     28.781773   
std        0.278952     24.095004     59.320681      0.581881     20.497410   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        1.000000     46.598618     15.856908      4.954439     14.000000   
50%        1.000000     62.756714     29.259776      5.324162     25.000000   
75%        1.000000     79.574249     53.347583      5.598760     39.000000   
max        2.000000    249.543213   2799.015591      6.610190    306.000000   

            difumin      Falla en  
count  63072.000000  63072.000000  
mean      43.453527      0.143550  
std       59.320681      0.568141  
min        0.000000      0.000000  
25%       15.856908      0.000000  
50%       29.259776      0.000000  
75%       53.347583      0.000000  
max     2799.015591      5.000000

In [16]:
cont = df['Filtro'].value_counts()
print(cont)

1    58052
0     3840
2     1180
Name: Filtro, dtype: int64


In [17]:
conte = df['Falla en'].value_counts()
print(conte)

0    58052
1     2553
2     1412
3      732
5      189
4      134
Name: Falla en, dtype: int64


In [18]:
df_fallas = df[df['Filtro'] == 0].copy()
df_buenas = df[df['Filtro'] == 1].copy()
df_recupera2 = df[df['Filtro'] == 2].copy()
df_perfe = df[df['Filtro'] >= 1].copy()

In [19]:
df_fallas

RutaArchivo  Filtro  resolucion  \
4      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   
22     D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   
23     D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   
24     D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   
25     D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   
...                                                  ...     ...         ...   
62429  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   
62430  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   
62431  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   
62432  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   
62877  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       0       False   

           brillo  contraste  entropia  p rostro    difumin  Falla en  
4      220.340332   0.699585  0.618864         0   0.699585         4  
22     204.447571   2.508508  3.762536         2   2.508508         3  
23     188.086670  13.068905  4.484253         0  13.068905         1  
24     133.491150  27.661770  5.410298         1  27.661770         1  
25     123.288940  23.500869  5.354915         1  23.500869         1  
...           ...        ...       ...       ...        ...       ...  
62429   76.805847   5.311855  5.474671         6   5.311855         1  
62430   76.326233   5.297214  5.471109         5   5.297214         1  
62431   76.831970   5.377063  5.472832         5   5.377063         1  
62432   74.315002   5.256193  5.427831         3   5.256193         2  
62877   74.053955   9.351213  4.954052         1   9.351213         1  

[3840 rows x 9 columns]

In [20]:
contefalla = df_fallas['Falla en'].value_counts()
print(contefalla)

1    1540
2    1245
3     732
5     189
4     134
Name: Falla en, dtype: int64


In [21]:
df_fallas.describe()

Filtro       brillo    contraste     entropia     p rostro  \
count  3840.0  3840.000000  3840.000000  3840.000000  3840.000000   
mean      0.0    46.907555     6.760513     3.961671     1.162760   
std       0.0    37.829505    46.417871     0.908335     1.883925   
min       0.0     0.000000     0.000000     0.000000     0.000000   
25%       0.0    22.214142     3.413141     3.614561     0.000000   
50%       0.0    33.676819     4.642990     4.193494     0.000000   
75%       0.0    63.360870     6.631521     4.469872     2.000000   
max       0.0   249.543213  2799.015591     6.451316    32.000000   

           difumin     Falla en  
count  3840.000000  3840.000000  
mean      6.760513     2.007031  
std      46.417871     1.083732  
min       0.000000     1.000000  
25%       3.413141     1.000000  
50%       4.642990     2.000000  
75%       6.631521     3.000000  
max    2799.015591     5.000000

In [22]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(df_fallas['RutaArchivo']).tolist()

# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

#     # Filtrar los registros correspondientes en el DataFrame
#     registros = df_fallas.loc[df_fallas['RutaArchivo'] == ruta]
#     # Mostrar los registros en la consola
#     print(registros)

In [23]:
df_buenas

RutaArchivo  Filtro  resolucion  \
0      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
1      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
2      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
3      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
5      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
...                                                  ...     ...         ...   
63067  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63068  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63069  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63070  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63071  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   

           brillo   contraste  entropia  p rostro     difumin  Falla en  
0       46.620056  168.688503  5.680590        47  168.688503         0  
1       47.367188  169.932763  5.672910        34  169.932763         0  
2       47.509460  180.040745  5.673102        46  180.040745         0  
3       95.777161  431.258760  6.025994        69  431.258760         0  
5      201.327698   12.006097  4.464224        12   12.006097         0  
...           ...         ...       ...       ...         ...       ...  
63067   67.594421   40.351175  5.329795        31   40.351175         0  
63068   66.825623   39.392251  5.306691        39   39.392251         0  
63069   66.991760   32.606774  5.312265        34   32.606774         0  
63070   66.380615   32.986551  5.344828        32   32.986551         0  
63071   66.631104   33.802185  5.361588        30   33.802185         0  

[58052 rows x 9 columns]

In [24]:
contebuena = df_buenas['Falla en'].value_counts()
print(contebuena)

0    58052
Name: Falla en, dtype: int64


In [25]:
df_buenas.describe()

Filtro        brillo     contraste      entropia      p rostro  \
count  58052.0  58052.000000  58052.000000  58052.000000  58052.000000   
mean       1.0     65.471374     46.590018      5.325885     31.125560   
std        0.0     22.217265     59.635307      0.430105     19.673213   
min        1.0     14.002197      5.702412      3.026166      4.000000   
25%        1.0     49.018524     18.654751      5.045465     17.000000   
50%        1.0     64.298553     32.079453      5.368412     27.000000   
75%        1.0     80.418945     56.212890      5.623086     41.000000   
max        1.0    243.900818   1412.857419      6.610190    306.000000   

            difumin  Falla en  
count  58052.000000   58052.0  
mean      46.590018       0.0  
std       59.635307       0.0  
min        5.702412       0.0  
25%       18.654751       0.0  
50%       32.079453       0.0  
75%       56.212890       0.0  
max     1412.857419       0.0

In [26]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(df_buenas['RutaArchivo']).tolist()

# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

#     # Filtrar los registros correspondientes en el DataFrame
#     registros = df_buenas.loc[df_buenas['RutaArchivo'] == ruta]
#     # Mostrar los registros en la consola
#     print(registros)

In [27]:
df_recupera2

RutaArchivo  Filtro  resolucion  \
126    D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       2       False   
127    D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       2       False   
129    D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       2       False   
130    D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       2       False   
238    D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       2       False   
...                                                  ...     ...         ...   
62416  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       2       False   
62417  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       2       False   
62418  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       2       False   
62419  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       2       False   
62420  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       2       False   

          brillo  contraste  entropia  p rostro    difumin  Falla en  
126    37.841797  10.578845  4.883450         3  10.578845         1  
127    35.899658  11.628370  4.873814         3  11.628370         1  
129    36.479248   9.573950  4.820503         1   9.573950         1  
130    36.919739   9.760131  4.850350         2   9.760131         1  
238    63.647949   6.145268  4.802205         3   6.145268         1  
...          ...        ...       ...       ...        ...       ...  
62416  71.510376   5.666175  5.337482         5   5.666175         1  
62417  71.274231   5.094656  5.313810         6   5.094656         1  
62418  72.981140   5.268371  5.351188         3   5.268371         2  
62419  73.029175   5.393647  5.371710         4   5.393647         1  
62420  73.204407   5.096267  5.354553         6   5.096267         1  

[1180 rows x 9 columns]

In [28]:
conterecu = df_recupera2['Falla en'].value_counts()
print(conterecu)

1    1013
2     167
Name: Falla en, dtype: int64


In [29]:
df_recupera2.describe()

Filtro       brillo    contraste     entropia     p rostro  \
count  1180.0  1180.000000  1180.000000  1180.000000  1180.000000   
mean      2.0    47.826691     8.556579     4.540055     3.354237   
std       0.0    27.822155     6.016661     0.386490     2.338585   
min       2.0    13.903687     4.512109     3.223778     1.000000   
25%       2.0    29.304764     5.464128     4.317993     2.000000   
50%       2.0    42.319702     6.742911     4.589052     3.000000   
75%       2.0    56.937592     9.334353     4.803716     3.000000   
max       2.0   149.061401    59.225337     5.507807    16.000000   

           difumin     Falla en  
count  1180.000000  1180.000000  
mean      8.556579     1.141525  
std       6.016661     0.348711  
min       4.512109     1.000000  
25%       5.464128     1.000000  
50%       6.742911     1.000000  
75%       9.334353     1.000000  
max      59.225337     2.000000

In [30]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(df_recupera2['RutaArchivo']).tolist()

# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

#     # Filtrar los registros correspondientes en el DataFrame
#     registros = df_recupera2.loc[df_recupera2['RutaArchivo'] == ruta]
#     # Mostrar los registros en la consola
#     print(registros)

In [31]:
df_perfe

RutaArchivo  Filtro  resolucion  \
0      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
1      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
2      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
3      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
5      D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
...                                                  ...     ...         ...   
63067  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63068  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63069  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63070  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   
63071  D:\sentirobots\Datasets\AFEW\Datos\Caras_AFEW\...       1       False   

           brillo   contraste  entropia  p rostro     difumin  Falla en  
0       46.620056  168.688503  5.680590        47  168.688503         0  
1       47.367188  169.932763  5.672910        34  169.932763         0  
2       47.509460  180.040745  5.673102        46  180.040745         0  
3       95.777161  431.258760  6.025994        69  431.258760         0  
5      201.327698   12.006097  4.464224        12   12.006097         0  
...           ...         ...       ...       ...         ...       ...  
63067   67.594421   40.351175  5.329795        31   40.351175         0  
63068   66.825623   39.392251  5.306691        39   39.392251         0  
63069   66.991760   32.606774  5.312265        34   32.606774         0  
63070   66.380615   32.986551  5.344828        32   32.986551         0  
63071   66.631104   33.802185  5.361588        30   33.802185         0  

[59232 rows x 9 columns]

In [32]:
conteperfe = df_perfe['Falla en'].value_counts()
print(conteperfe)

0    58052
1     1013
2      167
Name: Falla en, dtype: int64


In [33]:
df_perfe.describe()

Filtro        brillo     contraste      entropia      p rostro  \
count  59232.000000  59232.000000  59232.000000  59232.000000  59232.000000   
mean       1.019922     65.119863     45.832329      5.310230     30.572309   
std        0.139732     22.477991     59.283087      0.443098     19.861831   
min        1.000000     13.903687      4.512109      3.026166      1.000000   
25%        1.000000     48.504898     18.008573      5.024984     16.000000   
50%        1.000000     63.889465     31.421733      5.357987     27.000000   
75%        1.000000     80.152588     55.514397      5.617938     40.000000   
max        2.000000    243.900818   1412.857419      6.610190    306.000000   

            difumin      Falla en  
count  59232.000000  59232.000000  
mean      45.832329      0.022741  
std       59.283087      0.166923  
min        4.512109      0.000000  
25%       18.008573      0.000000  
50%       31.421733      0.000000  
75%       55.514397      0.000000  
max     1412.857419      2.000000

In [35]:
df.to_csv('D:\sentirobots\Datasets\AffWild2\Calidad-Rostros-AFEW.csv', index=False)

In [ ]:
###############################################################################

In [ ]:
# def ver_puntos_cara(lista):
#     if len(lista) > 0:
#         ultimo_elemento = lista[-1]  # Obtener el último elemento de la lista
#         if ultimo_elemento is not None and ultimo_elemento > 0:
#             return True
#     return False

In [ ]:
# #para mostrar las graficas HOG de rostros

# import matplotlib.pyplot as plt
# import cv2
# from skimage.feature import hog
# from skimage import exposure

# def detectar_rostros_hog(ruta_imagen):
#     # Cargar la imagen utilizando la ruta proporcionada
#     image = cv2.imread(ruta_imagen)

#     # Convertir la imagen a RGB (si está en formato BGR)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#     # Calcular el descriptor HOG
#     fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
#                         cells_per_block=(1, 1), visualize=True, multichannel=True, channel_axis=2)

#     # Crear una figura con dos subfiguras para mostrar la imagen original y el HOG
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)

#     ax1.axis('off')
#     ax1.imshow(image, cmap=plt.cm.gray)
#     ax1.set_title('Imagen original')

#     # Reescalar el histograma para una mejor visualización
#     hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

#     ax2.axis('off')
#     ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
#     ax2.set_title('Histograma de gradientes orientados (HOG)')

#     plt.show()

In [ ]:
# #DF PARA FALLAS
# ima={"RutaArchivo": imagenes}
# dfallas= pd.DataFrame(ima)
# dfallas["Carpeta base"] = carpeta_base1
# dfallas["Archivo"] = n1
# dfallas["Tipo de falla"] = lista_fallas
# dfallas["puntos"] = puntosdet
# dfallas["brillo"] = brillo
# dfallas["contraste"] = contr
# dfallas["resolucion"] = reso
# dfallas["entropia"] = entro
# dfallas

In [ ]:
# conteo = dfallas['Tipo de falla'].value_counts()
# print(conteo)

In [ ]:
# df_agrupado = dfallas.groupby(['Carpeta base', 'Archivo']).agg(lambda x: x.fillna(method='ffill').iloc[-1])
# df_agrupado

In [ ]:
# df_agrupado['no nulos'] = df_agrupado[['puntos', 'brillo', 'contraste', 'resolucion', 'entropia']].notnull().sum(axis=1)
# df_agrupado

In [ ]:
# conteo = df_agrupado['Tipo de falla'].value_counts()
# print(conteo)

In [ ]:
# conte = df_agrupado['no nulos'].value_counts()
# print(conte)

In [ ]:
# df_agrupado.describe()

In [ ]:
# dfresolucion = df_agrupado[pd.to_numeric(df_agrupado['resolucion'], errors='coerce').notnull()]
# dfresolucion

In [ ]:
# conteo0 = dfresolucion['resolucion'].value_counts()
# print(conteo0)

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfresolucion['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

In [ ]:
# dfpuntos = df_agrupado[pd.to_numeric(df_agrupado['puntos'], errors='coerce').notnull()]
# dfpuntos

In [ ]:
# dfpuntos.describe()

In [ ]:
# conteo1 = dfpuntos['puntos'].value_counts()
# print(conteo1)

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfpuntos['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfpuntos['RutaArchivo']).tolist()

# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

#     # Filtrar los registros correspondientes en el DataFrame
#     registros = dfpuntos.loc[dfpuntos['RutaArchivo'] == ruta]
#     # Mostrar los registros en la consola
#     print(registros)

In [ ]:
# dfbrillo = df_agrupado[pd.to_numeric(df_agrupado['brillo'], errors='coerce').notnull()]
# dfbrillo

In [ ]:
# dfbrillo.describe()

In [ ]:
# from collections import Counter
# valores_unicos = dfbrillo['brillo'].round()
# valores_unicos = sorted(valores_unicos, reverse=True)
# conteo = Counter(valores_unicos)
# conteo

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfbrillo['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

    
#     #Descartar bajo 14.5 y entran todas las caras

In [ ]:
# dfcontraste = df_agrupado[pd.to_numeric(df_agrupado['contraste'], errors='coerce').notnull()]
# dfcontraste

In [ ]:
# dfcontraste.describe()

In [ ]:
# valores_unicos = dfcontraste['contraste'].unique()
# valores_unicos = sorted(valores_unicos, reverse=True)
# valores_unicos

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfcontraste['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()
    
#     #3.5 hacia arriba es aceptable

In [ ]:
# dfentropia = df_agrupado[pd.to_numeric(df_agrupado['entropia'], errors='coerce').notnull()]
# dfentropia

In [ ]:
# dfentropia.describe()

In [ ]:
# conteo5 = dfentropia['entropia'].value_counts()
# print(conteo5)

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfentropia['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()
#     #esta bien como está

In [ ]:
####################################################################################################################

In [ ]:
# #PARA LA DETECCION REAL
# import os
# import pandas as pd
# import numpy as np
# file=[]
# #directorio_inicial = 'D:\sentirobots\Datasets\AFEW\Datos\Train_AFEW\AlignedFaces_LBPTOP_Points\AlignedFaces_LBPTOP_Points\Ejercicio entropia' # Carpeta actual
# directorio_inicial = 'D:\sentirobots\Datasets\AFEW\Datos\Train_AFEW\AlignedFaces_LBPTOP_Points\AlignedFaces_LBPTOP_Points\Faces' # Carpeta actual
# # Creamos una lista vacía para almacenar los nombres de archivo
# archivos = []
# calidad=[]
# carpeta_base=[]
# puntos=[]
# keypo=[]
# filtro=[]
# # Recorremos todos los archivos y carpetas a partir del directorio inicial
# for raiz, carpetas, archivos_en_carpeta in os.walk(directorio_inicial):
#     # Iteramos por cada archivo en la carpeta actual y lo agregamos a la lista de archivos
#     for archivo in archivos_en_carpeta:
#         # Verificamos si el elemento actual es un archivo
#         file.append(archivo)
#         if os.path.isfile(os.path.join(raiz, archivo)):
#             ruta_completa = os.path.join(raiz, archivo)
#             carpet, archi = os.path.split(ruta_completa)
#             c= os.path.basename(carpet)
#             carpeta_base.append(c)
#             archivos.append(ruta_completa)
#             if evaluate_image_quality(ruta_completa): # si la función retorna True
#                 cal=1
#                 calidad.append(cal) # agregamos un 1 a la lista
#             else: # si la función retorna False
#                 cal=0
#                 calidad.append(cal) # agregamos un 0 a la lista
#             if detect_faces(ruta_completa)>=5:
#                 p=1
#                 puntos.append(p)
#             else:
#                 p=0
#                 puntos.append(p)
#             if detect_faces(ruta_completa)>=5 and cal==1 or detect_faces(ruta_completa)>=5 and cal==0:
#                 filtro.append(1)
#             if  detect_faces(ruta_completa)<5 and cal==0 or detect_faces(ruta_completa)<5 and cal==1:
#                 filtro.append(0)
#             #print(len(filtro))
#             #print(len(puntos))
#             keypo.append(detect_faces(ruta_completa))

In [ ]:
# archiv={"RutaArchivo": archivos}
# df= pd.DataFrame(archiv)
# df["Carpeta final"] = carpeta_base
# df["Archivo"] = file
# df["Calidad"] = calidad
# df["Puntos"] = puntos
# df["KeyPoints"] = keypo
# df["Filtro"] = filtro

In [ ]:
# df

In [ ]:
# # Creamos un DataFrame vacío donde agregaremos las filas que cumplan la condición
# dfpuntoscriticos = pd.DataFrame(columns=['Filtro'])

# # Iteramos sobre cada fila del DataFrame original
# for index, row in df.iterrows():
#     # Comparamos si las dos columnas son iguales
#     #if row['Calidad'] != row['Puntos']:
#     if row['Filtro'] == 0:
#         # Si son iguales, agregamos la fila completa al DataFrame de resultado
#         dfpuntoscriticos = pd.concat([dfpuntoscriticos, row.to_frame().T], ignore_index=True)

# # Imprimimos el DataFrame de resultado
# dfpuntoscriticos

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# lista_ruta_criticos = np.array(dfpuntoscriticos['RutaArchivo']).tolist()
# for ruta in lista_ruta_criticos:
#     # Utilizar la función os.path.split() para separar la ruta en la última carpeta y el nombre del archivo
#     carpeta, archivo = os.path.split(ruta)
#     # Cargar la imagen utilizando la función imread() de matplotlib.image
#     imagen = mpimg.imread(ruta)
#     # Mostrar la imagen utilizando la función imshow() de matplotlib.pyplot con cmap=gray
#     plt.imshow(imagen, cmap='gray')
#     # Añadir el título de la imagen utilizando el nombre de la última carpeta y el nombre del archivo
#     plt.title(os.path.basename(carpeta) + '/' + archivo)
#     plt.show()

In [ ]:
# df.describe()

In [ ]:
# import os
# import cv2
# import numpy as np
# #NO USAR PORQUE ESTARÍA ASUMIENDO QUE SOLAMENTE HAY UNA PERSONA EN CADA CARPETA, SE PUEDE USAR PARA CONTAR CADA VEZ QUE HAY UN IDA Y VUELTA EN CAMARA O ESTIMAR CUANTAS PERSONAS HAY POR CARPETA. 
# def listar_archivos(directorio, archivos=None, coseno=[], usar=[]):
#     # Si no se proporcionó una lista, crear una vacía
#     if archivos is None:
#         archivos = []
#     # Inicializar la ruta de img1 en None
#     img1_ruta = None
#     # Recorrer todas las carpetas y archivos del directorio
#     for nombre_archivo in os.listdir(directorio):
#         # Unir el directorio y el nombre del archivo para obtener la ruta completa
#         ruta_archivo = os.path.join(directorio, nombre_archivo)
#         # Si es un archivo, guardar su ruta y nombre en la lista y comparar con el archivo anterior
#         if os.path.isfile(ruta_archivo):
#             archivos.append(ruta_archivo)
#             # Comprobar si el archivo anterior y el actual están en la misma carpeta
#             if len(archivos) > 1:
#                 carpeta_anterior = os.path.dirname(archivos[-2])
#                 carpeta_actual = os.path.dirname(archivos[-1])
#                 a1 = os.path.join(archivos[-2])
#                 a2 = os.path.join(archivos[-1])
#                 if carpeta_anterior == carpeta_actual:
#                     # Aquí puedes poner la acción que quieres que se haga
#                     if img1_ruta is None or cos_sim >= 0.85:
#                         img1_ruta = a1
#                     comparar = img1_ruta
#                     img1 = cv2.imread(comparar, cv2.IMREAD_GRAYSCALE)
#                     img2 = cv2.imread(a2, cv2.IMREAD_GRAYSCALE)

#                     # Aplicar un resize a las imágenes new_width=128, new_height=128
#                     img1 = cv2.resize(img1, (128, 128))
#                     img2 = cv2.resize(img2, (128, 128))

#                     # Aplanar las imágenes y normalizarlas
#                     img1 = img1.flatten() / 255.0
#                     img2 = img2.flatten() / 255.0

#                     # Calcular el producto punto entre ambas imágenes y las normas de cada una
#                     dot_product = np.dot(img1, img2)
#                     norm_img1 = np.linalg.norm(img1)
#                     norm_img2 = np.linalg.norm(img2)

#                     # Calcular la similitud del coseno entre ambas imágenes
#                     cos_sim = dot_product / (norm_img1 * norm_img2)
#                     print(cos_sim)
#                     coseno.append(cos_sim)
#                     if cos_sim<0.85:#0 es que no se puede usar
#                         usar.append(0)
#                     else:#1 es que se puede usar
#                         usar.append(1)
#                 else:
#                     coseno.append(1)
#                     usar.append(1)
#                     print(ruta_archivo)
#                     # Restablecer la ruta de img1 a None para la próxima carpeta
#                     img1_ruta = None
#             else:
#                 coseno.append(1)
#                 usar.append(1)
#                 print(ruta_archivo)
#                 # Restablecer la ruta de img1 a None para la próxima carpeta
#                 img1_ruta = None
#         # Si es una carpeta, recorrer sus archivos y guardarlos en la lista
#         elif os.path.isdir(ruta_archivo):
#             listar_archivos(ruta_archivo, archivos=archivos, coseno=coseno, usar=usar)
#         print(len(archivos))
#         print(img1_ruta)
#         print(len(coseno))
#     return archivos, coseno, usar

# # Ejemplo de uso
# directorio_inicial = 'D:\sentirobots\Datasets\AFEW\Datos\Train_AFEW\AlignedFaces_LBPTOP_Points\AlignedFaces_LBPTOP_Points\Faces' # Carpeta actual
# archivos = listar_archivos(directorio_inicial)


In [ ]:
# #evaluate_image_quality
# import cv2
# from skimage import io, img_as_float
# from skimage.filters import rank
# from skimage.morphology import disk
# import numpy as np
# def evaluate_image_quality(a):
#     # Cargar la imagen
#     #print(a)
#     img = cv2.imread(a)
#     # Verificar la resolución de la imagen
#     height, width, channels = img.shape
#     if height < 100 or width < 100:
#         #print("Falla hw")
#         return False

#     # Calcular la iluminación promedio de la imagen
#     hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     h, s, v = cv2.split(hsv)
#     mean_brightness = v.mean()
#     if mean_brightness < 14.5:
#         #print("Falla brillo")
#         #print(mean_brightness)
#         return False

#     # Calcular el contraste promedio de la imagen
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     mean_contrast = cv2.Laplacian(gray, cv2.CV_64F).var()
#     if mean_contrast < 3.5:
#         #print("mean contraste")
#         #print(mean_contrast)
#         #print("Falla contraste")
#         return False
    
#     # Calcular la entropía de la imagen
#     im1 = img_as_float(io.imread(a, as_gray=True))
#     radius = 15
#     selem = disk(radius)
#     entropy = rank.entropy(im1, selem=selem)
#     mean_entropy = np.mean(entropy)
#     # Evaluar entropia
#     if mean_entropy < 3:
#         #print("Falla entropia")
#         #print(mean_entropy)
#         return False
    
#     # Si todos los criterios se cumplen, la imagen es de buena calidad
#     return True 